In [6]:
import time
import numpy as np 
import pandas as pd 
from IPython.display import clear_output
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d   ## smoother
from tqdm.notebook import tqdm



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.preprocessing import MinMaxScaler
        
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau


/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/test_features.csv


In [7]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'

In [8]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [9]:
def pre_process_data(train_features, train_targets_scored):
    
    feature_columns = list(train_features.columns[1:])
    target_columns = list(train_targets_scored.columns[1:])
    
    removal_list = ['cp_type', 'cp_dose']
    for x in removal_list:
        feature_columns.remove(x)
        
    train_cat = train_features.merge(train_targets_scored, on='sig_id')
    train_cat = train_cat.select_dtypes(exclude=['object'])
    
    dummy_df = train_cat.loc[:, train_cat.columns != 'sig_id']
    df_float = dummy_df.astype(float)
#     scaler = MinMaxScaler()
#     df_float_scaled = pd.DataFrame(scaler.fit_transform(df_float), columns = df_float.columns)
#     df_float_scaled['sig_id'] = train_features['sig_id']
    return(df_float)

In [10]:
df_float_scaled = pre_process_data(train_features, train_targets_scored)
feature_columns = list(train_features.columns[1:])
removal_list = ['cp_type', 'cp_dose']
for x in removal_list:
    feature_columns.remove(x)
target_columns = list(train_targets_scored.columns[1:])

In [11]:
class TrainDataset(Dataset):
    def __init__(self, df, feature_columns, target_columns):
        
        self.features  = df[feature_columns].values
        self.targets = df[target_columns].values
        
    def sizes(self):
        print("features size = ", self.features.shape[1])
        print("targets size = ", self.targets.shape[1])

        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx]).float()
        target = torch.tensor(self.targets[idx]).float()
        
        return feature,target

In [19]:
full_dataset = TrainDataset(df_float_scaled, feature_columns, target_columns)

train_size = int(0.9 * len(full_dataset))  ## 90/10 split
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers = 8)

val_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle = True, num_workers = 8)

print(len(train_loader), "batches ")
print(len(val_loader), " batches ")

335 batches 
38  batches 


In [20]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
        
def show_lr(learning_rates):
    plt.plot(learning_rates, label = "learning rate")
    plt.ylabel("Learning rate", fontsize = 15)
    plt.grid(True)
    plt.legend()
    plt.show()
    
def show_deltas(deltas):
    deltas = gaussian_filter1d(deltas, sigma=3)
    plt.plot(deltas, "r", label = "Absolute error from label ")
    plt.ylabel("Error", fontsize = 15)
    plt.grid(True)
    plt.legend()
    plt.show()
    
def show_losses(loss_list):
    plt.plot(loss_list, 'r', label = 'Loss')
    plt.ylabel("Loss", fontsize = 15)
    plt.grid(True)
    plt.legend()
    plt.show()
    
def train_step(x, y, model, optimizer, criterion):
    optimizer.zero_grad()
    pred = model(x.to(device))
    y = y.float()
    loss = criterion(pred,y.to(device))
    loss.backward()
    optimizer.step()
    return loss.item()

In [35]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
    elif classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        
        

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(873)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(873, 2048))
        
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(2048, 1048))
        
        self.batch_norm3 = nn.BatchNorm1d(1048)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1048, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = (self.dense3(x))
        
        return x
    
    
    
model = Model()
model = model.to(device)
model.apply(weights_init)
print(model)




Model(
  (batch_norm1): BatchNorm1d(873, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=873, out_features=2048, bias=True)
  (batch_norm2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=2048, out_features=1048, bias=True)
  (batch_norm3): BatchNorm1d(1048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (dense3): Linear(in_features=1048, out_features=206, bias=True)
)


In [36]:
losses = []
val_losses = []
learning_rates = []
average_deltas = []
val_corr=[]

optimizer = optim.Adam(model.parameters(), lr=0.8e-3)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min', 
                                                 factor=0.8, 
                                                 patience=3, 
                                                 eps=1e-4, 
                                                 verbose=True)
criterion = nn.BCEWithLogitsLoss()


In [ ]:
num_epochs=20
for epoch in range(num_epochs):
    
    total_loss = 0
    total_correct = 0
    total_loss2 = 0
    total_correct2 = 0
    
    print ("epoch ", epoch+1, " out of ", num_epochs )

    model.train()
    for batch in tqdm(train_loader, desc = " Training batches : "):
        (x_batch, y_batch) = batch
        loss = train_step(x_batch.to(device), y_batch.to(device), model, optimizer, criterion)
        losses.append(loss)
    scheduler.step(1.)   ## lr decay caller 
    learning_rates.append(get_lr(optimizer))
    
    
    with torch.no_grad():
        model.eval()

        for x_val, y_val in tqdm(val_loader, desc = "running on test set --"):
            yhat =model(x_val.to(device))  # pred 
            val_loss = criterion(yhat.to(device), y_val.to(device))
            val_losses.append(torch.mean(val_loss.view(-1)).item())  ## metrics 



    clear_output(wait = True)

    show_lr(learning_rates)
    show_losses(val_losses)
    show_losses(losses)
    #show_deltas(average_deltas)

epoch  1  out of  50


In [17]:
min(val_losses)

0.0006734605995006859

In [18]:
min(val_losses) 
## old = 0.010247811675071716 
## with leaky = 0.010694394819438457  
# with weights init = 0.00914179626852274
# with better LRscheduler = 0.0083483150228858, 0.007986623793840408
## all run = 0.002
## unscaled data = 0.0006

0.0006734605995006859

In [ ]:
plt.plot(val_losses[39:])
plt.grid()

In [ ]:
test_df = pre_process_data(test_features, submission )
test_df.head()

In [ ]:
test_dataset = TrainDataset(test_df, feature_columns, target_columns)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False, num_workers = 8)

In [ ]:
list_of_preds=[]
with torch.no_grad():
    model.eval()
    for x_test, y_test in tqdm(test_loader, desc = "running on test set --"):
        pred =model(x_test.to(device, dtype=torch.float))  # pred 
        pred = pred.cpu()
        pred = pred.sigmoid()
        list_of_preds.append(list(pred[0].numpy()))

In [ ]:
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')
sub_cp = submission
sub_cp.to_csv('./submission_cp.csv', index=None, header=True)

In [ ]:
import csv 
a = list_of_preds  
with open('./submission_cp.csv', "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(a)

In [ ]:
final_sub = pd.read_csv('./submission_cp.csv', header = None)
final_sub.head()

In [ ]:
final_sub.columns = submission.columns[1:]
final_sub["sig_id"] = submission["sig_id"]


In [ ]:
good_cols = np.roll(final_sub.columns.values, 1)
final_sub = final_sub[good_cols]

In [ ]:
final_sub.to_csv("./submission.csv", index=False)
final_sub